# week 10: more poisson processes: birth-death and lineage diversification

## eeb463

## tomo parins-fukuchi

**how often do things happen?**

- amino acid substitutions along a protein
- speciation events in a large clade
- fossil deposition events

**how often do things happen?**

- typically might imagine a **rate**:
    - substitution rate
    - speciation rate
    - fossilization rate

**how often do things happen?**

- typically might imagine a **rate**:
    - substitution rate: how many substitutions occur per million years?
    - speciation rate: how many new species form per million years?
    - fossilization rate: how many fossils are preserved per million years?

**this offers us a simple approach to simulation**

- discretize time into steps
- check to see if an event happened, given rate, &lambda;



In [ ]:
# how many substitutions occur in a protein after 1000 time steps?

import random
import matplotlib.pyplot as plt

n_subs = 0
stt = [n_subs]
rate = 0.05
for i in range(999):
    r = random.random()
    if r < rate:
        n_subs += 1
    stt.append(n_subs)

plt.plot([i for i in range(1000)], stt,"o")
plt.xlabel("time step")
plt.ylabel("# substitutions")
plt.show()

**how many events can we expect in time window $t$?**

In [ ]:
subs = []
for _ in range(500):
    n_subs = 0
    rate = 0.05
    for i in range(1000):
        r = random.random()
        if r < rate:
            n_subs += 1
    subs.append(n_subs)

plt.hist(subs)
plt.xlabel("# substitutions")
plt.ylabel("count")
plt.show()

**poisson distribution**

- discrete probability dist
- 1 param:
  - &lambda; -- how often do events happen?


**poisson pmf**

- probability of _k_ events in an interval, $t$:

$$\frac{(\lambda t)^ke^{-\lambda t}}{k!}$$

In [ ]:
def factorial(n):
    nums = [i for i in range(n,1,-1)]
    prod = 1
    for i in nums:
        prod *= i
    return prod

In [ ]:
import math

def poisson_pmf(lam, t, k):
    num = ((lam*t)**k) * math.exp(-(lam*t))
    denom = factorial(k)
    return num / denom

probs = []
x = []
for i in range(30,70):
    probs.append(poisson_pmf(0.05,1000,i))
    x.append(i)
    
plt.plot(x,probs,"o")
plt.xlabel("# substitutions")
plt.ylabel("probability")
plt.show()


**how often do things happen?**
   - how long do we need to wait between events?

**poisson process**

- counts (\# events) over time follow a poisson distribution
- waiting times between events are _exponentially_ distributed


**exponential**

  - lopsided distribution
  - single parameter: "rate" (aka "lambda")



**exponential**

- cumulative density function:

![h:500 center](images/exponential_cdf.svg)



**exponential**

- probability density function:

![h:500 center](images/exponential_pdf.svg)


In [ ]:
import numpy as np

def exp_pdf(lam, dt):
    dens = lam * math.exp(-lam * dt)
    return dens

rates = [.2, .4, .6]
for rate in rates:
    probs = []
    x = np.linspace(0,15,50)
    for i in x:
        probs.append(exp_pdf(rate, i))
    plt.plot(x,probs,"-")
    
plt.xlabel("waiting time length (dt)")
plt.ylabel("density")
plt.show()

In [ ]:
# collect waiting times between mutations

import random
import matplotlib.pyplot as plt
import numpy as np

n_subs = 0
w = []
curw = 0
rate = 0.05
for i in range(5000):
    r = random.random()
    if r < rate:
        n_subs += 1
        w.append(curw)
        curw = 0
    else:
        curw += 1

plt.hist(w)
plt.axvline(x=np.mean(w),color="black")
plt.text(70,100,"mean # steps: "+str(round(np.mean(w),2)))
plt.xlabel("# time steps spent waiting")
plt.ylabel("frequency")
plt.show()

**how often do things happen?**
- expect a substitution to occur about once every 20 time steps given &lambda; = 0.05
    - empirically, things seem to happen about every $ \frac{1}{\lambda} $ steps

**poisson process**
  - let's refer to the rate for the poisson processs as $\lambda_p$
  - times between events are exponential dist with:
      - rate: $\lambda_p $
      - mean waight time: $ \mu = \frac{1}{\lambda_p}$

**waiting times**

does this expectations actually align with what we see in the world?

**waiting times**

![bg right h:700](images/earthquakes.png)

**waiting times**

soccer goals (FIFA WC 1990-2000)

**waiting times**

![bg right h:700](images/goals.png)

**waiting times**

- american football scoring (GB packers 2023 season)

**waiting times**

![bg right h:700](images/all_pack.png)

**waiting times**

![bg right h:700](images/opp_scores.png)

**waiting times**

![bg right h:700](images/pack_scores.png)

**waiting times**

- exponentially distributed waiting times often a shockingly good fit
- occasionally less so
    - extension of basic poisson process may be more appropriate

**how can we simulate a poisson process?** 

- we've simulated a poisson process by:
    1. dividing a time frame into discrete slices
    2. checking to see if an event happened in each slice, with prob &lambda;

- there is a better way

**"gillespie" or "doob-gilespie" algorithm**

- general approach is to draw waiting times from an exponential distribution
  - determine when the "next" event is


**simulating exponential**

- _u_ is a random number between 0 and 1
- _rate_ is our exponential rate parameter

$$ \frac{\log(1-u)}{-rate} $$


**simulating exponential**

```
draw = random()
rate = 1        # can be any val > 0

r = math.log(1-draw) / (-rate)
```


In [ ]:
import distributions as dist

lam = 0.05 # this will be our poisson process rate
exp = dist.exponential(lam)

curtime = 0.
nsub = 0
times = [curtime]
subs = [nsub]
while True:
    w = exp.sample(1)[0]
    curtime += w
    if curtime > 1000.:
        break
    times.append(curtime)
    nsub += 1
    subs.append(nsub)



In [ ]:
plt.plot(times,subs,"o")
plt.xlabel("time")
plt.ylabel("# substitutitons")
plt.show()

**poisson processes**

- there is a lot we can do with this


**poisson processes**

- there is a lot we can do with this
- let's quickly revisit our example from last week

**poisson processes**

![bg right h:700](images/ctmc.svg)

In [ ]:
def set_Q(gain,loss):
    Q = np.array(
        [ 
        [-gain, gain], 
        [loss, -loss] 
        ])
    return Q

In [ ]:
from scipy.linalg import expm
import random
import matplotlib.pyplot as plt

def sim_discrete(Q, time_window, dt = 0.1):
    char_state = 0
    states = [char_state]
    curtime = 0.
    times = [curtime]
    while curtime < time_window:
        curtime += dt
        P = expm(Q*dt)
        cur_row = P[char_state]  # we'll pull out the row corresponding to our current state
        prob_change = cur_row[abs(1 - char_state)]  # find probability of change by pulling index opposite of char_state
        r = random.random()
        if r < prob_change:
            char_state = abs(1 - char_state)  # swap current character state to other
        states.append(char_state)
        times.append(curtime)
    return times, states

In [ ]:
Q = set_Q(0.2,0.1)
times,states = sim_discrete(Q, 20.)
plt.plot(times,states)
plt.show()

**poisson processes**

- in the approach above, we evenly spaced _dt_ and checked if a change occurred every _dt_

In [ ]:
Q = set_Q(0.2,0.1)
times,states = sim_discrete(Q, 20.)
plt.plot(times,states,"o")
plt.show()

**poisson processes**

- in the approach above, we evenly spaced _dt_ and checked if a change occurred every _dt_
- we can instead find all _dt_ intervals where a change happens by drawing waiting times from an exponential distribution
    - we will get the rate ($\lambda$) parameter value from _Q_

In [ ]:
from scipy.linalg import expm
import random
import matplotlib.pyplot as plt

def sim_discrete_gillespie(Q, time_window):
    char_state = 0
    states = [char_state]
    curtime = 0.
    times = [curtime]
    off_diag = Q[~np.eye(Q.shape[0],dtype=bool)]
    event_rate = sum(off_diag)
    wait_dist = dist.exponential(Q[0][1])
    while True:
        wait = wait_dist.sample(1)[0]
        curtime += wait
        if curtime > time_window:
            times.append(time_window)
            states.append(char_state)
            break
        char_state = abs(1 - char_state)
        wait_dist.rate = Q[char_state][abs(1 - char_state)]  # need to change the rate in case they are assymetrical
        times.append(curtime)
        states.append(char_state)
    return times, states

In [ ]:
Q = set_Q(0.2,0.1)
times,states = sim_discrete_gillespie(Q,20)
plt.plot(times,states,"o")
plt.show()

**poisson processes**

- last week, we sampled the process at evenly spaced _dt_ intervals
- now, we can sample only the _dt_ values that span substitution events
    - this saves us a tremendous amount of computation

## 5 minute break

![bg right h:700](images/trilo_bud1.svg)

![bg right h:700](images/trilo_bud2.svg)

![bg right h:700](images/trilo_bud3.svg)

![bg right h:700](images/trilo_bud4.svg)

![bg right h:700](images/trilo_bud5.svg)

![bg right h:700](images/trilo_bud6.svg)

![bg right h:700](images/trilo_bud7.svg)

![bg right h:700](images/trilo_bud8.svg)

**birth-death process**

- models like this tell us a lot about how we might expect lineages to diversify




**birth-death process**

- models like this tell us a lot about how we might expect lineages to diversify
- species stochastically:
    - arise (birth)
    - go extinct (death)



**birth-death process**

- also useful for modeling:
    - population growth
    - disease outbreak dynamics
    - lifespan of 18th century english surnames


**birth and death**

  - population increments by integers according to params
    - birth (_b_ aka &lambda;)
    - death (_d_ aka &mu;)

  - each birth or death is referred to as an 'event'
      - we can model this as a poisson process
  - if population size reaches 0, simulation is over


**birth and death**

we assume that:
  - this process is _markovian_
  - events occur one at a time



**birth and death**

  - simplest version is a "pure birth" process
      - b > 0
      - d = 0
  - this is identical to our protein substitution example


**birth and death**

  - where b != d:
    - waiting times between events are exponentially distributed 
        - rate = b + d


**birth and death**

  - where b != d:
    - waiting times between events are exponentially distributed 
        - rate = b + d
    - probability of an event being a birth:
        - $\frac{b} { (b+d)}$
    - probability of an event being a death:
        - $\frac{d} { (b+d)}$


In [ ]:
import distributions as dist

b = 0.025
d = 0.025
lam = b + d
exp = dist.exponential(lam)

curtime = 0.
n = 10
times = [curtime]
pop_size = [n]
while True:
    w = exp.sample(1)[0]
    curtime += w
    if curtime > 1000. or n == 0:
        break
    times.append(curtime)
    if b / (b + d) > random.random():
        n += 1
    else:
        n -= 1
    pop_size.append(n)

In [ ]:
plt.plot(times,pop_size)
plt.xlabel("time")
plt.ylabel("num. species")
plt.show()

**birth-death process**

- this process can be used to generate a stochastic distribution of trends

In [ ]:
def sim_bd_reps(b, d, time_window = 1000000., nrep = 100):
    alltimes = []
    pop_sizes = []
    for _ in range(nrep):
        curtime = 0.
        n = 10
        times = [curtime]
        pop_size = [n]
        while True:
            w = exp.sample(1)[0]
            curtime += w
            if curtime > time_window or n == 0:
                break
            times.append(curtime)
            if b / (b + d) > random.random():
                n += 1
            else:
                n -= 1
            pop_size.append(n)
        alltimes.append(times)
        pop_sizes.append(pop_size)
    return alltimes,pop_sizes
    

In [ ]:
alltimes, pop_sizes = sim_bd_reps(0.5, 0.5, 10000., 10)

for i in range(len(alltimes)):
    times = alltimes[i]
    pop_size = pop_sizes[i]
    plt.plot(times,pop_size)

plt.xlabel("time")
plt.ylabel("num species")
plt.show()

In [ ]:
alltimes, pop_sizes = sim_bd_reps(0.02, 0.03)

for i in range(len(alltimes)):
    times = alltimes[i]
    pop_size = pop_sizes[i]
    plt.plot(times,pop_size)

plt.xlabel("time")
plt.ylabel("num species")
plt.show()

In [ ]:
alltimes, pop_sizes = sim_bd_reps(0.03, 0.02)

for i in range(len(alltimes)):
    times = alltimes[i]
    pop_size = pop_sizes[i]
    plt.plot(times,pop_size)

plt.xlabel("time")
plt.ylabel("num species")
plt.show()

**birth-death process**

   - this approach assumes that births and deaths happen at the same rate regardless of pop size
   - we might expect more events to occur when there are more individuals present
   - need to reparameterize as _per-capita_ rates

In [ ]:
def sim_bd_per_capita_reps(b, d, time_window = 1000000., nrep = 100, start_n = 10):
    alltimes = []
    pop_sizes = []
    for _ in range(nrep):
        curtime = 0.
        n = start_n
        times = [curtime]
        pop_size = [n]
        exp = dist.exponential((b + d) * n)
        while True:
            w = exp.sample(1)[0]
            curtime += w
            if curtime > time_window:
                break

            if b / (b + d) > random.random():
                n += 1
            else:
                n -= 1
                
            times.append(curtime)
            pop_size.append(n) 
            exp.rate = (b + d) * n
            if n == 0:
                break

        alltimes.append(times)
        pop_sizes.append(pop_size)
    return alltimes,pop_sizes

**birth-death process**

- this version of the model ensures diversification is a multiplicative process

In [ ]:
starting_size = 100

alltimes, pop_sizes = sim_bd_per_capita_reps(0.004, 0.003, 2000, 10, starting_size)

for i in range(len(alltimes)):
    times = alltimes[i]
    pop_size = pop_sizes[i]
    plt.plot(times,pop_size)

plt.xlabel("time")
plt.ylabel("num species")
plt.show()

**birth-death process**

- the growth curve is a consequence of the _net diversification rate_: $b - d$
    - higher net div. rate -> faster multiplicative growth in species diversity

**birth-death process**

- starting size of the clade also strongly determines dynamics

In [ ]:
starting_size = 1

alltimes, pop_sizes = sim_bd_per_capita_reps(0.005, 0.003, 1000, 10, starting_size)

for i in range(len(alltimes)):
    times = alltimes[i]
    pop_size = pop_sizes[i]
    plt.plot(times,pop_size)

plt.xlabel("time")
plt.ylabel("num species")
plt.show()

**birth-death process**

- n = 0 is an absorbing state
- given infinite time, all trajectories will go extinct

In [ ]:
starting_size = 10

alltimes, pop_sizes = sim_bd_per_capita_reps(0.003, 0.003, 1000, 10, starting_size)

for i in range(len(alltimes)):
    times = alltimes[i]
    pop_size = pop_sizes[i]
    plt.plot(times,pop_size)

plt.xlabel("time")
plt.ylabel("num species")
plt.show()

**birth-death process**

- model makes assumptions about the probability of extinction throughout duration of a species
    - specifically, a species is always at equal risk of extinction
 - let's verify that this assumption behaves as we expect using our simulations
 - can keep track of each individual species from the simulation, along w/ its speciation and extinction time

In [ ]:
def sim_bd_survivorship(b, d, time_window = 1000000., start_n = 10):
    species = {}
    curtime = 0.
    for i in range(start_n):
        species["species"+str(i)] = [0.]
    extant_sp = list(species.keys())
    exp = dist.exponential((b + d) * float(len(extant_sp)))

    while True:
        w = exp.sample(1)[0]
        curtime += w
        if curtime > time_window:
            for sp in species:
                duration = species[sp]
                if len(duration) == 1:
                    species[sp].append(time_window)
            break

        if b / (b + d) > random.random():
            newsp = "species" + str(len(species))
            species[newsp] = [curtime]
            extant_sp.append(newsp)
        else:
            killsp = random.choice(extant_sp)
            species[killsp].append(curtime)
            extant_sp.remove(killsp)

        exp.rate = (b + d) * float(len(extant_sp))
        if len(extant_sp) == 0:
            break

    return species

**birth-death process**

- first thing we can do is visualize the distribution of species durations

In [ ]:
ranges = sim_bd_survivorship(0.002,0.002,10000,50)

durations = []
for i in ranges:
    ran = ranges[i]
    duration = ran[1] - ran[0]
    durations.append(duration)
plt.hist(durations)
plt.xlabel("duration")
plt.ylabel("count`")
plt.axvline(np.median(durations),color="black")
plt.show()

**birth-death process**

what happens if we increase the rates?

In [ ]:
ranges = sim_bd_survivorship(0.009,0.009,10000,50)

durations = []
for i in ranges:
    ran = ranges[i]
    duration = ran[1] - ran[0]
    durations.append(duration)
plt.hist(durations)
plt.xlabel("duration")
plt.ylabel("count`")
plt.axvline(np.median(durations),color="black")
plt.show()

**birth-death process**

- temporal scale is a consequence of the _turnover rate:_ $b + d$
- higher turnover = shorter durations

**birth-death process**

- next, let's look at the survivorship curve

In [ ]:
durations = np.array(durations)
times = np.linspace(0.,max(durations)+0.1,20)
surv = []

for i in times:
    n_surv = list(durations < i).count(False)
    p_surv = float(n_surv) / len(durations)
    surv.append(p_surv)


plt.plot(times,surv)
plt.semilogy(times, surv) 
plt.ylabel("prop. species surviving")
plt.xlabel("duration")
plt.show()

**birth-death process**

- another way of thinking about this is that we combine two separate poisson processes
    - one for speciation, one for extinction
- waiting times between extinctions are exponentially distributed
    - highest density is always on an immediate extinction
- but risk of extinction for an individual species does not change throughout its duration
    - there is some empirical support for this

**birth-death process**


![bg right h:700](images/survivorship.svg)

**birth-death process**

- "Van Valen's law":
    - biological taxa are always at a constant risk of extinction
    - explained this pattern using the "Red Queen" model


**birth-death process**

> Now, here, you see, it takes all the running you can do just to keep in the same place. If you want to get somewhere else, you must run at least twice as fast!

Red Queen from _Through the Looking Glass_

**birth-death process**

- 'Red Queen' model:
    - biotic interactions exert constant extinction pressure against taxa
    - taxa adapt to better meet these pressures, but their antagonists do as well
    - their risk of extinction remains constant


**birth-death process**

- these ideas have prompted much exploration into the processes that might explain the pattern
- most of this research has been rooted in birth-death and related processes
    - these questions have demanded approaches for inference under birth-death models


**birth-death process**

- likelihood of speciation events

![bg right h:700](images/phylogeny_model_bds_n_desc.svg)


**birth-death process**

- how often does speciation happen?
- probability of a species giving rise to $k$ descendants over time $t$ given rate &lambda;

$$P(n | t, \lambda ) = \frac{(\lambda t)^ke^{-\lambda t}}{k!}$$


**birth-death process**

- likelihood of extinction events

![bg right h:700](images/phylogeny_model_bds_extinction_t.svg)


**birth-death process**

- how often does extinction happen?
- probability of waiting time $t$ until extinction given extinction rate &mu;

$$P(e_t | t, \mu ) = \mu e^{-\mu t}$$


**birth-death process**

- can derive likelihood functions using these equations to:
    - fit birth-death model parameters that best explain fossil dataset

**birth-death process**


![bg right h:700](images/birth_death_likelihood.png)


**birth-death process**

- can even fit more complex models where rates vary through time or among lineages

**birth-death process**

![bg right h:700](images/foote_div.png)



**birth-death process**

- can derive likelihood functions using these equations to:
    - fit birth-death model parameters that best explain fossil dataset
    - fit survivorship curves based on parameter estimates


**birth-death process**

![bg right h:700](images/trilobite_survivorship.png)



**birth-death process**

> Using a time-homogeneous branching model, it is estimated that trilobite genera and species originating during the Ordovician survived three times longer than Cambrian genera and species... Some of the ultimate causes of these differences in survivorship include (1) taxonomic inconsistency, (2) greater environmental stability in the Ordovician, and (3) more highly structured ecosystems in the Ordovician that may have led to the weeding out of extinction-prone taxa.

Foote 1988, _Paleobiology_, "Survivorship analysis of Cambrian and Ordovician trilobites" 

**birth-death process**

- VV's law is consistent with a birth-death model
- what if Van Valen was wrong?
- can alternative stochastic processes better explain what we see?


**birth-death process**

![bg right h:700](images/survivorship_raup.png)


**birth-death process**

- sometimes, variations of the standard birth-death process explain better the patterns we see


**birth-death process**

![bg right h:700](images/hubbell.svg)



**birth-death process**

- Hubbell's neutral theory (remember the forest fires from week 3) is based on a birth-death model
- examines how stochastic changes in relative abundance of lineages can lead to extinction
    - contains terms for community size, reproductive rate, speciation rate

**birth-death process**


![bg right h:700](images/neutral.png)


**birth-death process**

- this is the case for patterns other than survivorship as well

**birth-death process**

- it may be more biologically realistic to expect an equilibrium diversity


![bg right h:700](images/sepkoski.png)